<a href="https://colab.research.google.com/github/psaw/hse-ai24-ml/blob/main/BVD_screencast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Разложение ошибки

In [ ]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

In [ ]:
RANDOM_STATE = 42

In [ ]:
data = fetch_california_housing(as_frame=True)

X = data.data
y = data.target

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size = 0.3, random_state = RANDOM_STATE)

In [ ]:
scaler = StandardScaler()
scaler.fit(Xtrain)

Xtrain = pd.DataFrame(data = scaler.transform(Xtrain), columns = X.columns)
Xtest = pd.DataFrame(data = scaler.transform(Xtest), columns = X.columns)

## Bias-variance decomposition

В библиотеке `mlxtend` есть функция `bias_variance_decomp` для оценки компонент разложения ошибки.

In [ ]:
%%capture
!pip install mlxtend --upgrade

### Гиперпараметры
* `estimator` - семейство моделей
* `X_train, y_train` - обучающие данные
* `X_test, y_test` - тестовые данные
* `loss` - функция потерь (MSE для регрессии, 0-1 loss (доля ошибок модели) для классификации)
* `num_rounds=200` - число случайных подвыборок из `X_train` для обучения модели

### Возвращаемые значения

* `avg_expected_loss` - ошибка на тестовых данных
* `avg_expected_bias` - смещение
* `avg_expected_variance` - разброс

In [ ]:
from mlxtend.evaluate import bias_variance_decomp

X_train = Xtrain.values
y_train = ytrain.values
X_test = Xtest.values
y_test = ytest.values

In [ ]:
avg_mse, avg_bias, avg_var = bias_variance_decomp(LinearRegression(), X_train, y_train,
                                                  X_test, y_test, loss = 'mse',
                                                  random_seed=np.random.seed(RANDOM_STATE))

In [ ]:
print('Loss:', avg_mse)
print('Bias:', avg_bias)
print('Variance:', avg_var)

Loss: 0.5311442880444522
Bias: 0.5282182136689957
Variance: 0.0029260743754565798


In [ ]:
avg_mse, avg_bias, avg_var = bias_variance_decomp(DecisionTreeRegressor(), X_train, y_train,
                                                  X_test, y_test, loss = 'mse',
                                                  random_seed=np.random.seed(RANDOM_STATE))

In [ ]:
print('Loss:', avg_mse)
print('Bias:', avg_bias)
print('Variance:', avg_var)

Loss: 0.5662872207084668
Bias: 0.2510460794697548
Variance: 0.31524114123871183


Мы видим, что решающее дерево гораздо точнее предсказывает целевую переменную, чем линейная регрессия  
(*bias* гораздо меньше), но при этом гораздо сильнее переобучено (*variance* гораздо больше).  
За счет этого суммарная ошибка у дерева чуть больше, чем у линейной регрессии.

Путем подбора гиперпараметров дерева можно снизить переобучение и суммарную ошибку.